In [12]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, TimeoutError, LocalCluster, progress
from dask.diagnostics import ProgressBar
import pandas as pd
import hvplot.dask
import re
from textblob import TextBlob
from langdetect import detect
from langdetect import detect_langs

#code found on stack exchange
#makes a new client if one is not running, runs 4 workers
try:
    client = Client('tcp://localhost:8787', timeout='5s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=6, ip='localhost')
    client = Client(cluster)


client

<Client: 'tcp://127.0.0.1:8787' processes=6 threads=12, memory=16.00 GiB>

In [13]:
df = dd.read_csv('mental_health_tweets.csv', sep='\t')

def preprocess(text):
    # remove mentions
    text = re.sub('@[a-zA-Z0-9]+', '', text)
    
    #  remove hashtag symbol
    text = re.sub('#', '', text)
    
    # remove retweet identifier
    text = re.sub('RT[\s]+', '', text)
    
    # remove links
    text = re.sub('https?:\/\/\S+', '', text)


    text = re.sub('[^a-zA-Z\\s]', '', text)

    text = text.strip()

    return text


df['text'] = df['tweet'].map(preprocess)
df = df.loc[df['text'] != '']

df['language'] = df['text'].map(lambda x: detect(x))
df = df.loc[df['language'] == 'en']

df.to_csv('mental_health_tweets_english.csv')

2022-11-29 00:50:52,069 - distributed.worker - WARNING - Compute Failed
Key:       ('map-087b8aa09d1b56bebf32dd5117ca3da0', 0)
Function:  map
args:      (0              acho q ele não vai passar de mais um trauma
1        @raygabieee Eu n consigo comer mt de uma vez p...
2        @JayMan471 I think that's a common psychosis i...
3        EVERYONE GETTING AUTISM USERS AND I CANT THINK...
4        of Tropical Depression Nicole.  #开云 #世界杯赔率 #炸金...
                               ...                        
53900    I need 16 more men for my mental health silenc...
53901    Mental health crises are excluded from some st...
53902    @hericaaaaaaaaa isso podia gerar até um trauma...
53903    @Xenoblade_PR Entire xc3 party can be split in...
53904    I ordered to @Flipkart on 05.11.2022. Order No...
Name: tweet, Length: 53905, dtype: object, <function <lambda> at 0x7fdd96597f70>, None)
kwargs:    {}
Exception: "LangDetectException('No features in text.')"



Exception: LangDetectException('No features in text.')